# World Happyness Report - The Course No1 Project


In [1]:
# Import usefull libraries for the project
import matplotlib.pyplot as plt
%matplotlib inline

## A. Importing, cleaning and numerical summaries

### Load and analyze the data

First, we build a pandas dataframe object by loading the data.csv file.

In [2]:
import os
import pandas as pd

datafile_path = os.path.join('data','data.csv')

df = pd.read_csv(datafile_path)


Now it's time to grab some informations from this freshly loaded dataframe.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 12 columns):
Country             153 non-null object
Happiness Rank      153 non-null int64
Happiness Score     153 non-null float64
Economy             153 non-null float64
Family              153 non-null float64
Health              153 non-null float64
Freedom             153 non-null float64
Generosity          153 non-null float64
Corruption          153 non-null float64
Dystopia            153 non-null float64
Job Satisfaction    151 non-null float64
Region              153 non-null object
dtypes: float64(9), int64(1), object(2)
memory usage: 14.4+ KB


This dataframe is made of a total of 12 columns and 153 lines.

The df.info() result confirm that there is no missing data in this dataste as all the columns contains exactly the same 'non-null' values as the total of lines in the dataset.
Columns types are floats (values of the observations), integer (for the country rank) and 2 generic types (Country names and Regions)


### Reindex the dataset

Displaying a few lines of the dataframe using a `df.head(5)` function call, we can see that index could be rearranged using the Country Name.
This can be easily done using the following code.

_Do not forget the `inplace=True` to update our dataframe_



In [4]:
df.set_index('Country',inplace=True)
df.head(3)

,Happiness Rank,Happiness Score,Economy,Family,Health,Freedom,Generosity,Corruption,Dystopia,Job Satisfaction,Region
Country,,,,,,,,,,,
Malta,27,6.527,1.343280,1.488412,0.821944,0.588767,0.574731,0.153066,1.556863,85.2,Western Europe
Zimbabwe,138,3.875,0.375847,1.083096,0.196764,0.336384,0.189143,0.095375,1.597970,56.3,Africa
Cyprus,65,5.621,1.355938,1.131363,0.844715,0.355112,0.271254,0.041238,1.621249,88.7,Eastern Europe


At that point, our dataframe contains 11 column as the `'Country'` one became its index.

In [7]:
df.columns

Index(['Happiness Rank', 'Happiness Score', 'Economy', 'Family', 'Health',
       'Freedom', 'Generosity', 'Corruption', 'Dystopia', 'Job Satisfaction',
       'Region'],
      dtype='object')

### Grab some stats

Basic statistics values of the data can be obtain using the `describe()` function.

We can quickly restrict the statistcs displayed to the mean, the min and the max values of each numerical columns using the `.loc()`function.

In [19]:
df.describe().loc[['mean','min','max']]

,Happiness Rank,Happiness Score,Economy,Family,Health,Freedom,Generosity,Corruption,Dystopia,Job Satisfaction
mean,78.169935,5.349281,0.982433,1.186630,0.550117,0.408489,0.245324,0.123179,1.853072,75.209934
min,1.000000,2.693000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.377914,44.400000
max,155.000000,7.537000,1.870766,1.610574,0.949492,0.658249,0.838075,0.464308,3.117485,95.100000


In [23]:
df.sort_values('Happiness Rank').tail(10)

,Happiness Rank,Happiness Score,Economy,Family,Health,Freedom,Generosity,Corruption,Dystopia,Job Satisfaction,Region
Country,,,,,,,,,,,
Yemen,146,3.593,0.591683,0.935382,0.310081,0.249464,0.104125,0.056767,1.345601,58.9,Asia-Pacific
South Sudan,147,3.591,0.397249,0.601323,0.163486,0.147062,0.285671,0.116794,1.879567,NaN,Africa
Liberia,148,3.533,0.119042,0.872118,0.229918,0.332881,0.266550,0.038948,1.673286,56.6,Africa
Guinea,149,3.507,0.244550,0.791245,0.194129,0.348588,0.264815,0.110938,1.552312,55.1,Africa
Togo,150,3.495,0.305445,0.431883,0.247106,0.380426,0.196896,0.095665,1.837229,44.8,Africa
Rwanda,151,3.471,0.368746,0.945707,0.326425,0.581844,0.252756,0.455220,0.540061,51.7,Africa
Syria,152,3.462,0.777153,0.396103,0.500533,0.081539,0.493664,0.151347,1.061574,62.7,Asia-Pacific
Tanzania,153,3.349,0.511136,1.041990,0.364509,0.390018,0.354256,0.066035,0.621130,57.8,Africa
Burundi,154,2.905,0.091623,0.629794,0.151611,0.059901,0.204435,0.084148,1.683024,54.3,Africa
